# 1.1 Experiment: market basket data

## A. Import modules
## B. Prepare functions
## C. Load & prepare data
## D. Load test data
## E. Experiment:
### 1) SVD
### 2) word2vec
### 3) Bitmap
### 4) Poincare
## F. Results


# A. Import modules and function

In [37]:
# import libs
import numpy as np
import time
import gensim
from collections import Counter
from itertools import combinations
import pandas as pd
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds
from math import log
from gensim.models import word2vec
from annoy import AnnoyIndex
import random
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter
import tensorflow
from keras.utils import to_categorical
import numpy as np
import math

results = []

# B. Prepare functions

In [55]:
def generate_svd(transactions_dict, patterns_dict, n_dim = 300, negative = False):
	data_list = [v for _,v in transactions_dict.items()]

	unigrams_cnt = Counter()
	bigrams_cnt = Counter()
	for text in data_list:
		for x in text:
			unigrams_cnt[x] += 1
		for x, y in map(sorted, combinations(text, 2)):
			bigrams_cnt[(x, y)] += 1

	id2uni = {}
	uni2id = {}
	it = 0

	for uni,_ in unigrams_cnt.items():
		id2uni[it] = uni
		uni2id[uni] = it
		it +=1


	sum_uni = float(sum(unigrams_cnt.values()))
	sum_bi = float(sum(bigrams_cnt.values()))

	data, rows, cols = [], [], []
	for (x, y), n in bigrams_cnt.items():
		rows.append(uni2id[x])
		cols.append(uni2id[y])
		data.append(log((n / sum_bi) / (unigrams_cnt[x] / sum_uni) / (unigrams_cnt[y] / sum_uni)))
	PMI = csc_matrix((data, (cols, rows)), shape = (len(unigrams_cnt), len(unigrams_cnt)))
	U,_,_ = svds(PMI, k = n_dim)
	norms = np.sqrt(np.sum(np.square(U), axis=1, keepdims=True))
	U /= np.maximum(norms, 1e-7)
    
    

	result_t_dict = {}
	result_p_dict = {}

	for key in transactions_dict.keys():
		for product in transactions_dict[key]:
			temp = [U[uni2id[product]] for product in transactions_dict[key]]
			result_t_dict[key] = power_means([x for x in temp])

	if not negative:        
		for key in patterns_dict.keys():
			result_p_dict[key] = power_means([U[uni2id[product]] for product in patterns_dict[key]])
	else:
		for key in patterns_dict.keys():
			result_p_dict[key] = power_means([U[uni2id[product]] for product in patterns_dict[key][:-1]])
			result_p_dict[key] = result_p_dict[key]- U[uni2id[patterns_dict[key][-1]]]
    
	return (result_t_dict,result_p_dict)

In [51]:
def generate_word2vec(transactions_dict, patterns_dict, n_dim = 300, n_workers = 10, n_epochs = 20, negative = False):
	data_list = [v for _,v in transactions_dict.items()]
	window_size = max([len(x) for x in data_list])

	model = word2vec.Word2Vec(data_list, size = n_dim, window = window_size, min_count = 1, workers = n_workers)

	model.train(data_list, total_examples = len(data_list), epochs = n_epochs)

    
	result_t_dict = {}
	result_p_dict = {}

	for key in transactions_dict.keys():
		result_t_dict[key] = power_means([model[product] for product in transactions_dict[key]])

	if not negative:        
		for key in patterns_dict.keys():
			result_p_dict[key] = power_means([model[product] for product in patterns_dict[key]])
	else:
		for key in patterns_dict.keys():
			result_p_dict[key] = power_means([model[product] for product in patterns_dict[key][:-1]])
			result_p_dict[key] = result_p_dict[key] - patterns_dict[key][-1]
            
	return (result_t_dict,result_p_dict)

In [40]:
def generate_answers(patterns_dict, transactions_dict):
    answers = {}
    l = len([x for x in patterns_dict.keys()])/2

    for i,key in enumerate(patterns_dict.keys()):
        
        if (i%l == 0):
            print(i/len([key for key in patterns_dict.keys()]),time.time()-a_time)
            
        answers[key] = [t_key for t_key in transactions_dict.keys() if set(patterns_dict[key][:-1]) <= set(transactions_dict[t_key]) and patterns_dict[key][-1] not in transactions_dict[t_key]]    
    
    return answers
    

In [41]:
def power_means(list_of_vectors, p = 1):
	data = np.array(list_of_vectors)

	return np.power(np.power(data,p).mean(axis=0), 1/p)

In [42]:

def generate_patterns(input_dict):
    output_dict = {}
    i = 0
    for key in input_dict.keys():
        for k in range(4):
            temp = combinations(input_dict[key],k)
            for y in temp:
                output_dict['p_'+str(i)] = [l for l in y]
                if output_dict['p_'+str(i)] == []:
                    output_dict.pop('p_'+str(i))
                
                i+=1
        
    
    return output_dict
    

def generate_transactions(input_dict):
    output_dict = {}
    random.seed(14231)
    i = 0
    for key1 in input_dict.keys():
        for item in input_dict[key1]:
            try:
                new_key = 'test_'+str(i)
                result = []
                temp = []



                key2 = random.sample(input_dict.keys(),1)[0]
                key3 = random.sample(input_dict.keys(),1)[0]
                temp.append(random.choice(input_dict[key2]))
                temp.append(random.choice(input_dict[key3]))
                it1 = np.random.choice(input_dict[key1])
                it2 = np.random.choice(input_dict[key1])
                temp.append(it1)
                temp.append(it2)
                np.random.shuffle(temp)
                result = temp[:2]+[item]+temp[2:4]
                result = list(set(result))
                output_dict[new_key] = result

                if output_dict[new_key] == []:
                    output_dict.pop(new_key)
            except:
                pass
            
            i+=1
    
    return output_dict


In [166]:

def add_zeros(input_list,l = 8):
    return input_list + [0 for i in range(l-len(input_list))]

def gen_index(input_data_dict,d=2):
    count = Counter([x for sublist in input_data_dict.values() for x in sublist])

    depth = int(np.ceil(math.log(len(count.keys()), d)))
    
    org_input = input_data_dict
    
    
    base_length = d**depth
    
    onehot_encoder = MultiLabelBinarizer()
    
    input_data_dict = {'p'+str(k): org_input[k][:-1] for k in org_input.keys() if k[0] == 'p'}
    input_data_dict.update({'p'+str(k): org_input[k] for k in org_input.keys() if k[0] == 't'})
    input_data_dict.update({'n'+str(k): [org_input[k][-1]] for k in org_input.keys() if k[0] == 'p'})
    result = onehot_encoder.fit_transform(input_data_dict.values())
    
    result = [[x[int(d*i):int(d*(i+1))] for i in range(int(int(base_length)/d))] for x in result]
        
    result = [[signature2int(add_zeros([int(z) for z in y],l=d)) for y in x] for x in result]
    
    if depth != 1:
        for l in range(len(result)):
            obj = result[l]
            for i in range(depth,1,-1):
                new_level = []
                old_level_length = d**(i-1)
                old_level = obj[:old_level_length]
                new_level_length = d**(i-2)
                new_level = [signature2int([int(bool(x)) for x in old_level[d*j:d*(j+1)]]) for j in range(int(old_level_length/d))]

                for k in range(new_level_length):
                    result[l].insert(k,new_level[k])
    
    
    index_p_dict = {}
    index_n_dict = {}
    result_p_dict = {}
    result_n_dict = {}
    
    for i,key in enumerate(input_data_dict.keys()):
        row_dict = {}
        k = 0 
        
        for j in range(len(result[i])):
            if result[i][j] != 0:
                row_dict[k] = j
                k+= 1
        
        if key[0] == 't':
            index_p_dict[key] = row_dict
            result[i] = [y for y in result[i] if y != 0]
            result_p_dict[key] = result[i]
        elif key[0] == 'n':
            key = key[1:]
            index_n_dict[key] = row_dict
            result[i] = [y for y in result[i] if y != 0]
            result_n_dict[key] = result[i]
        elif key[0] == 'p':
            key = key[1:]
            index_p_dict[key] = row_dict
            result[i] = [y for y in result[i] if y != 0]
            result_p_dict[key] = result[i]
        
    return (result_p_dict, result_n_dict, index_p_dict, index_n_dict)    


def signature2int(signature):

     return int(''.join(map(str, signature)), 2)
    


def skip(i_t,i_p,i):
    skip = 0
    while i_t[i+skip] != i_p[i]:
        skip += 1
    return skip   


def search(pattern,p_index,transaction,t_index):
    if transaction[0] & pattern[0] == pattern[0]:
        for i in range(1,len(pattern)):
            p = skip(t_index,p_index,i)
            if transaction[i+p] & pattern[i] != pattern[i]:
                return False
    else:
        return False
    return True 

# C. Load data

In [187]:
a_time = time.time()
data_df = pd.read_csv('order_products__train.csv', nrows = 5000)

data_dict = {}

for index,row in data_df.iterrows():
    if row['order_id'] not in data_dict.keys():
        data_dict[row['order_id']] = [str(row['product_id'])]
    else:
        data_dict[row['order_id']].append(str(row['product_id']))


print("Processing took {} s".format(time.time()-a_time))

Processing took 0.5668094158172607 s


# D. Generate test data

In [188]:
a_time = time.time()

patterns_dict = generate_patterns(data_dict)

patterns_1p1n = {key:patterns_dict[key] for key in patterns_dict.keys() if len(patterns_dict[key]) == 2}
p_length = len([key for key in patterns_1p1n.keys()])

patterns_2p1n = {}
i = 0
for key in patterns_dict.keys():
    if len(patterns_dict[key]) == 3:
        patterns_2p1n[key] = patterns_dict[key]
        i += 1
    if i == p_length:
        break

transactions_dict = generate_transactions(data_dict)

answers_1p1n = generate_answers(patterns_1p1n, transactions_dict)
answers_2p1n = generate_answers(patterns_2p1n, transactions_dict)

print("Processing took {} s".format(time.time()-a_time))

0.0 1.6096723079681396
0.0 203.00218391418457
Processing took 401.7210669517517 s


# E. Experiment

## 1) SVD

In [189]:
a_time = time.time()
svd_dim = 500


(t_svd, p_svd) = generate_svd(transactions_dict, patterns_dict, n_dim = svd_dim, negative = True)

print("Processing took {} s".format(time.time()-a_time))

# Last run time:
# 3) nrows = 50000 - 900 s / cut off = 0.37

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice.
  after removing the cwd from sys.path.


Processing took 20.573729991912842 s


In [190]:
a_time = time.time()

t2int = {}
int2t = {}

for i,k in enumerate(transactions_dict.keys()):
    t2int[k] = i
    int2t[i] = k

annoy_index = AnnoyIndex(svd_dim,"angular")

for key in transactions_dict.keys():
    annoy_index.add_item(t2int[key],t_svd[key])

annoy_index.build(1000)

print("Processing took {} s".format(time.time()-a_time))

Processing took 6.648879766464233 s


In [191]:
a_time = time.time()

correct_answers = 0
answers_counter = 0 

for y in patterns_1p1n.keys():

    x = annoy_index.get_nns_by_vector(p_svd[y],n = len(answers_1p1n[y]))
    correct_answers += len([z for z in answers_1p1n[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_1p1n[y])


results.append({'timestamp':time.time(),'algorithm':'SVD','dimensions':svd_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':1})

In [192]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_2p1n.keys():

    x = annoy_index.get_nns_by_vector(p_svd[y],n = len(answers_2p1n[y]))
    correct_answers += len([z for z in answers_2p1n[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_2p1n[y])
    
results.append({'timestamp':time.time(),'algorithm':'SVD','dimensions':svd_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':2})

## 2) word2vec

In [193]:
a_time = time.time()

word2vec_dim = 500

(t_word2vec, p_word2vec) = generate_word2vec(transactions_dict, patterns_dict, n_dim = word2vec_dim)

print("Processing took {} s".format(time.time()-a_time))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Processing took 28.785987615585327 s


In [194]:
a_time = time.time()

t2int = {}
int2t = {}

for i,k in enumerate(transactions_dict.keys()):
    t2int[k] = i
    int2t[i] = k

annoy_index = AnnoyIndex(word2vec_dim,"angular")

for key in transactions_dict.keys():
    annoy_index.add_item(t2int[key],t_word2vec[key])

annoy_index.build(1000)

print("Processing took {} s".format(time.time()-a_time))

Processing took 9.11513352394104 s


In [195]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_1p1n.keys():

    x = annoy_index.get_nns_by_vector(p_word2vec[y],n = len(answers_1p1n[y]))
    correct_answers += len([z for z in answers_1p1n[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_1p1n[y])

results.append({'timestamp':time.time(),'algorithm':'word2vec','dimensions':word2vec_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':1})

In [196]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_2p1n.keys():

    x = annoy_index.get_nns_by_vector(p_word2vec[y],n = len(answers_2p1n[y]))
    correct_answers += len([z for z in answers_2p1n[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_2p1n[y])

results.append({'timestamp':time.time(),'algorithm':'word2vec','dimensions':word2vec_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':2})

## 3) Bitmap index

In [197]:
a_time = time.time()

temp_dict = patterns_dict
temp_dict.update(transactions_dict)

t_p_index,t_n_index,i_p_index,i_n_index = gen_index(temp_dict, d = 64)

print("Index generated")

print("Processing took {} s".format(time.time()-a_time))

Index generated
Processing took 1139.9963972568512 s


In [198]:
a_time = time.time()


test_dict = {}
test_i = {}
pattern_p_dict = {}
pattern_n_dict = {}
pattern_p_i = {}
pattern_n_i = {}

# test_keys = [key for key in data_dict.keys() if str(key)[0] == 't']

for key in t_p_index.keys():
    if str(key)[0] == 't':
        test_dict[key] = t_p_index[key]
        test_i[key] = i_p_index[key]
    else:
        pattern_p_dict[key] = t_p_index[key]
        pattern_p_i[key] = i_p_index[key]
        pattern_n_dict[key] = t_n_index[key]
        pattern_n_i[key] = i_n_index[key]

            
        

print("Processing took {} s".format(time.time() - a_time))

Processing took 1.179093837738037 s


In [199]:
a_time = time.time()

for key in patterns_1p1n.keys():
    k = len(answers_1p1n[key])
    i = 0
    for t_key in test_p_dict.keys():
        if search(pattern_p_dict[key],pattern_p_i[key],test_dict[t_key],test_i[t_key]) and not search(pattern_n_dict[key],pattern_n_i[key],test_dict[t_key],test_i[t_key]):
            i += 1            
            if i == k:
                break
        
results.append({'timestamp':time.time(),'algorithm':'bitmap','dimensions':'X','time':time.time()-a_time,'quality':1, 'n_patterns':p_length,'patterns_length':2})

In [200]:
a_time = time.time()

for key in patterns_2p1n.keys():
    k = len(answers_2p1n[key])
    i = 0
    for t_key in test_dict.keys():
        if search(pattern_p_dict[key],pattern_p_i[key],test_dict[t_key],test_i[t_key]) and not search(pattern_n_dict[key],pattern_n_i[key],test_dict[t_key],test_i[t_key]):
            i += 1            
            if i == k:
                break
        
results.append({'timestamp':time.time(),'algorithm':'bitmap','dimensions':'X','time':time.time()-a_time,'quality':1, 'n_patterns':p_length,'patterns_length':3})

## 4) Poincare

In [215]:
a_time = time.time()

poincare_embeddings = pd.read_csv("market_data_poincare_A_10k_100d.tsv", sep="\t",header=None)
poincare_dim = 50

poincare_rels_dict = {}

for _,row in poincare_embeddings.iterrows():
    poincare_rels_dict[str(row[0])[:-2]] = [row[x] for x in range(1,poincare_dim+1)]

    
patterns_poincare = {}
for key in patterns_dict.keys():
    if key != '':
        patterns_poincare[key] = power_means([poincare_rels_dict[obj] for obj in patterns_dict[key] if obj != ''])
    
    if patterns_poincare[key] == []:
        patterns_poincare.pop(key)
    if str(patterns_poincare[key]) == 'nan':
        patterns_poincare.pop(key)
    patterns_poincare[key] -= poincare_rels_dict[patterns_dict[key][-1]]


transactions_poincare = {}
for key in transactions_dict.keys():
    if key != '':
        transactions_poincare[key] = power_means([poincare_rels_dict[obj] for obj in transactions_dict[key] if obj != ''])
    
    if transactions_poincare[key] == []:
        transactions_poincare.pop(key)
    if str(transactions_poincare[key]) == 'nan':
        transactions_poincare.pop(key)
    transactions_poincare[key] -= poincare_rels_dict[transactions_dict[key][-1]]
        
print("Processing took {} s".format(time.time()-a_time))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Processing took 201.519464969635 s


In [216]:
a_time = time.time()

t2int = {}
int2t = {}

for i,k in enumerate(transactions_dict.keys()):
    t2int[k] = i
    int2t[i] = k

annoy_index = AnnoyIndex(poincare_dim,"angular")

for key in transactions_dict.keys():
    annoy_index.add_item(t2int[key],transactions_poincare[key])

annoy_index.build(1000)

print("Processing took {} s".format(time.time()-a_time))

Processing took 4.356727600097656 s


In [217]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_1p1n.keys():

    x = annoy_index.get_nns_by_vector(patterns_poincare[y],n = len(answers_1p1n[y]))
    correct_answers += len([z for z in answers_1p1n[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_1p1n[y])

results.append({'timestamp':time.time(),'algorithm':'poincare','dimensions':poincare_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':1})

In [218]:
a_time = time.time()

correct_answers = 0
answers_counter = 0

for y in patterns_2p1n.keys():

    x = annoy_index.get_nns_by_vector(patterns_poincare[y],n = len(answers_2p1n[y]))
    correct_answers += len([z for z in answers_2p1n[y] if z in [int2t[o] for o in x]])
    answers_counter += len(answers_2p1n[y])


results.append({'timestamp':time.time(),'algorithm':'poincare','dimensions':poincare_dim,'time':time.time()-a_time,'quality':correct_answers/answers_counter, 'n_patterns':p_length,'patterns_length':2})

# E. Results

In [423]:
import json 

for line in results:
    if (json.loads(json.dumps(line))['algorithm']) == 'bitmap':
        print(line)

{'timestamp': 1570562800.1420643, 'algorithm': 'bitmap', 'dimensions': 'X', 'time': 0.6713907718658447, 'quality': 1, 'n_patterns': 1000, 'patterns_length': 1}
{'timestamp': 1570562800.7061377, 'algorithm': 'bitmap', 'dimensions': 'X', 'time': 0.5577149391174316, 'quality': 1, 'n_patterns': 1000, 'patterns_length': 2}
{'timestamp': 1570562801.3265765, 'algorithm': 'bitmap', 'dimensions': 'X', 'time': 0.6139039993286133, 'quality': 1, 'n_patterns': 1000, 'patterns_length': 3}


In [219]:
results_df = pd.DataFrame(results)
results_df['mat'] = results_df.apply(lambda row: row['time']/row['n_patterns'],axis = 1) #mean answer time
results_df

,algorithm,dimensions,n_patterns,patterns_length,quality,time,timestamp,mat
0,SVD,20,8723,1,0.305887,14.318589,1.570707e+09,0.001641
1,SVD,20,8723,2,0.056903,1.242591,1.570707e+09,0.000142
2,SVD,200,8723,1,0.885860,7.793822,1.570707e+09,0.000893
3,SVD,200,8723,2,0.735541,1.168597,1.570707e+09,0.000134
4,word2vec,100,8723,1,0.659217,11.258839,1.570713e+09,0.001291
5,word2vec,100,8723,2,0.708955,1.434071,1.570713e+09,0.000164
6,word2vec,10,8723,1,0.169152,15.979667,1.570713e+09,0.001832
7,word2vec,10,8723,2,0.043843,1.539035,1.570713e+09,0.000176
8,word2vec,50,8723,1,0.528661,10.717786,1.570713e+09,0.001229
9,word2vec,50,8723,2,0.575093,1.097180,1.570713e+09,0.000126


In [220]:
results_df.to_csv('experiment1B_results.csv',index=None)